In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **COMPX525 Assignment 1 - Benedict Johnson 1349629**

> **Referenced Code Files**
> * https://www.kaggle.com/code/duythanhng/birdclef-2022-keras-simple-tutorial
> * https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
> *https://numpy.org/doc/stable/reference/generated/numpy.reshape.html
> *https://www.kaggle.com/code/foolishboi/birdclef22-playing-with-data

# **1. Import all necessary modules**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import json 
import ast
import sklearn
import random
import soundfile as sf
import librosa
import librosa.display
import IPython.display as ipd
import plotly.express as px
import soundfile as sf
import descartes
import geopandas as gpd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow.keras.backend as K
import tensorflow as tf
import math
import sys
import gc

from tensorflow.keras.applications import EfficientNetB5
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model, Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from math import ceil
from numpy import array

# **2. EfficientNet-B5 Model Parameters**

In [ ]:
chosen_buffer = 5 #5s
row = random.randint(0,100)
data_frames = []
num_rows = 216
num_columns = 216
num_channels = 1
n_mels = 512

chosen_test = 0.2 #20% test, %80 training
epochs = 50 #2 hours training time
chosen_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
chosen_activation = 'sigmoid'
chosen_loss = 'categorical_crossentropy'
chosen_metrics = ["accuracy"]
batch_number = 32 #Low to ensure enough memory from system

#  **3. Define Audio Functions**

In [ ]:
path = '/kaggle/input/birdclef-2022/'
os.listdir(path)
with open(path+'scored_birds.json') as f:
    scored_birds = json.load(f)
def read_ogg_file(path, file):
    data, samplerate = sf.read(path+file)
    return data, samplerate
def wave_function(data, samplerate):
    sr = samplerate
    fig = plt.figure(figsize=(8, 4))
    x = range(len(data))
    y = data
    plt.plot(x, y)
    plt.legend(loc='upper center')
    plt.grid()

# **4. Loading Data**

In [ ]:
path = '/kaggle/input/birdclef-2022/'
os.listdir(path)
train = pd.read_csv(path+'train_metadata.csv')
labels = list(train['primary_label'].unique())
test = pd.read_csv(path+'test.csv')
tax_data = pd.read_csv(path+'eBird_Taxonomy_v2021.csv')
sub_csv = pd.read_csv(path+'sample_submission.csv')
#Submission Soundscape
os.listdir(path+'test_soundscapes')

#  **5. Understanding The Data**

> **Display number of bird classes**

In [ ]:
print("Total number of bird species is {:,}! ".format(len(train['common_name'].unique())))
#Display ratings values.
plt.figure(figsize=(15, 6))
ax = sns.countplot(train['rating'], palette="Spectral")
plt.title("Class Audio Ratings (float 0.0-5.0)", fontsize=15)
plt.ylabel("Samples", fontsize=14)
plt.xlabel("Ratings Value", fontsize = 14)
plt.xticks(rotation=90, fontsize=12)
plt.yticks(fontsize=12)


>**Display number of bird sound types**

In [ ]:
z = train['type'].apply(lambda x: x[2:-2].split("', '")).reset_index().explode("type")
z = z['type'].apply(lambda x: x.strip().lower()).reset_index()
z['type'] = z['type'].replace('calls', 'call')
t = list(z['type'].value_counts().head(15).reset_index()['index'])
index_data = z[z['type'].isin(t)]
plt.figure(figsize=(15, 6))
ax = sns.countplot(index_data['type'], palette="Spectral", order = index_data['type'].value_counts().index)
plt.title("Most-Frequent Bird Sound Types", fontsize=15)
plt.ylabel("Occurances", fontsize=14)
plt.xlabel("Bird Sound Type", fontsize=14)
plt.yticks(fontsize=12)
plt.xticks(rotation=90, fontsize=12)



> **Random Sample File**

In [ ]:
train.iloc[row]
filename = train.loc[row, 'filename']
sample_label = train.loc[row, 'primary_label']
filename.split('/')[1] in os.listdir(path+'train_audio/'+sample_label)
data, samplerate = sf.read(path+'train_audio/'+filename)

In [ ]:
row

> **Visualizing Random Sample File**

In [ ]:
wave_function(data, samplerate)

 # **6. Attempted Implementation of One-Hot Encoding**

In [ ]:
encoder = LabelEncoder()
def one_hot_encoding(labels):
    encoded_int = encoder.fit_transform(array(labels))
    encoded_int = encoded_int.reshape(len(encoded_int),1)
    encoding_one_hot = OneHotEncoder(sparse=False) #Returns array sparse = False
    return encoding_one_hot.fit_transform(encoded_int)
ohe_labels = one_hot_encoding(labels)
ohe_labels

# **7. Data Preperation**

In [ ]:
def sample_audio(file_path, is_save):
    filename = file_path.replace("/", "_")
    file_path = "/kaggle/input/birdclef-2022/train_audio/" + file_path
    audio, sr = librosa.load(file_path)
    buffer = chosen_buffer * sr
    samples_total = len(audio)
    samples_wrote = 0
    counter = 1
    audio_split = []
    audio_filenames = []
    while samples_wrote < samples_total:
        #check if the buffer is not exceeding total samples 
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote

        block = audio[samples_wrote : (samples_wrote + buffer)]
        audio_split.append(block)

        # Write 5 second segment
        if is_save == True:
            out_filename = "/kaggle/working/each5s/split_" + str(counter) + "_" + filename
            audio_filenames.append(out_filename)
            sf.write(out_filename, block, sr)
        counter += 1
        samples_wrote += buffer
    return audio_split, sr, audio_filenames

def audio_split(_df):
    data = []
    for index, row in _df.iterrows():
        sample_audio(row["filename"], True)
        audio_lst, sr, filenames = sample_audio(row["filename"], True)
        for idx, y in enumerate(audio_lst):
            data.append([row["primary_label"], row["filename"], filenames[idx]])

    dataframe = pd.DataFrame(data, columns=['primary_label', 'original_filename', 'filename'])
    dataframe.to_csv("/kaggle/working/dataframe.csv", index=False)

In [ ]:
data_frames = []
for label in labels:
    tmp_df = train[train["primary_label"] == label].sample(n=1, replace=True).reset_index(drop=True)
    data_frames.append(tmp_df)
sample_df = pd.concat(data_frames).reset_index(drop=True)
sample_df

In [ ]:
!mkdir -p "/kaggle/working/each5s"
audio_split(sample_df)

In [ ]:
dataframe = pd.read_csv("/kaggle/working/dataframe.csv")
dataframe

# **8. Training/Inferencing EfficientNet-B5**

> **Setup Functions**

In [ ]:
def features(y, sr):
    feat = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=num_rows, n_mels=n_mels)
    if feat.shape[1] <= num_columns:
        pad_width = num_columns - feat.shape[1]
        feat = np.pad(feat, pad_width=((0,0),(0,pad_width)), mode='constant')
    return feat

In [ ]:
class process_data(Sequence):
    def __init__(self,
                _X,
                batch_size=32,
                n_channels=1,
                n_columns=470,
                n_rows=120,
                shuffle=True):
        self.batch_size = batch_size
        self.X = _X
        self.n_channels = n_channels
        self.n_columns = n_columns
        self.n_rows = n_rows
        self.shuffle = shuffle
        self.img_indexes = np.arange(len(self.X))
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.img_indexes) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temps = [self.img_indexes[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temps)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temps):
        X = np.empty((self.batch_size, self.n_rows, self.n_columns))
        y = np.empty((self.batch_size), dtype=int)
        for i, ID in enumerate(list_IDs_temps):
            file_path = self.X.iloc[ID]["filename"]
            audio, sr = librosa.load(file_path)
            feat = features(audio, sr)
            x_features = feat.tolist()
            label = self.X.iloc[ID]["target"]
            X[i] = np.array(x_features)
            y[i] = label
        X = X.reshape(X.shape[0], self.n_rows, self.n_columns, self.n_channels)
        
        return X, to_categorical(y, num_classes=len(labels))

> **Setup EfficientNet-B5 Model & Training**

In [ ]:
parameters = dict(batch_size=batch_number,n_rows=num_rows,n_columns=num_columns,n_channels=num_channels)
parameters_train = dict(shuffle=True,**parameters)
parameters_valid = dict(shuffle=False,**parameters)
dataframe['target'] = dataframe['primary_label'].apply(lambda x: labels.index(x))
dataframe

In [ ]:
def efficientNet_model():
    model_input = Input(shape=(num_rows, num_columns, num_channels))
    model_conc = Concatenate()([model_input, model_input, model_input])
    base_model = EfficientNetB5(include_top=False, weights='../input/d/datasets/aeryss/keras-pretrained-models/EfficientNetB5_NoTop_ImageNet.h5', input_tensor=model_conc)
    avgpool = GlobalAveragePooling2D()(base_model.output)
    outputs = Dense(len(labels), activation=chosen_activation)(avgpool)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model


In [ ]:
def train_efficientNet_model(model, train_gen, val_gen):
    checkpoint_model_path = "/kaggle/working/EfficientNetB5.h5"
    metric = "val_accuracy"
    model.compile(optimizer=chosen_optimizer, loss=chosen_loss, metrics=chosen_metrics)
    num_epochs = epochs
    checkpointer = ModelCheckpoint(filepath=checkpoint_model_path,monitor=metric, verbose=1, save_best_only=True)
    es_callback = EarlyStopping(monitor=metric, patience=5, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor=metric, factor=0.3, patience=1, verbose=1, min_delta=0.0001, cooldown=1, min_lr=0.00001)
    fitted_model = model.fit(train_gen,epochs=num_epochs,validation_data=val_gen,callbacks=[checkpointer,es_callback,reduce_lr],verbose=1)
    #Plot training and validation scores
    plt.figure(1, figsize = (15,10))
    plt.subplot(221)
    plt.plot(fitted_model.history['loss'])
    plt.plot(fitted_model.history['val_loss'])
    plt.title('EfficientNet-B5 Loss vs Number of Epochs')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Training EfficientNet-B5 Loss', 'EfficientNet-B5 Validation Loss'])
    plt.subplot(222)
    plt.plot(fitted_model.history['accuracy'])
    plt.plot(fitted_model.history['val_accuracy'])
    plt.title('EfficientNet-B5 Accuracy vs Number of Epochs')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Training EfficientNet-B5 Accuracy', 'EfficientNet-B5 Validation Accuracy'])
    plt.show()

In [ ]:
X_train, X_valid, _, _ = train_test_split(dataframe, dataframe["target"], test_size=chosen_test, random_state=42)
train_generator = process_data(X_train, **parameters_train)
valid_generator = process_data(X_valid, **parameters_valid)
cnn_model = efficientNet_model()
train_efficientNet_model(cnn_model, train_generator, valid_generator)

# **9. Load Model & Submission**

In [ ]:
pred = {'row_id': [], 'target': []}

#Setup Model
cnn_model = efficientNet_model()
#Load trained model weights!
cnn_model.load_weights("./EfficientNetB5.h5")
dataframe = pd.read_csv("./dataframe.csv")
labels = list(dataframe['primary_label'].unique())
test_path = "/kaggle/input/birdclef-2022/test_soundscapes/"
files = [f.split('.')[0] for f in sorted(os.listdir(test_path))]
birds_path = "/kaggle/input/birdclef-2022/scored_birds.json"
with open(birds_path) as bf:
    birds = json.load(bf)

sample_submission = pd.read_csv('../input/birdclef-2022/sample_submission.csv')


for f in files:
    file_path = test_path + f + '.ogg'
    audio, sr = librosa.load(file_path)
    # Get number of samples for 5 seconds; replace 5 by any number
    buffer = 5 * sr
    samples_total = len(audio)
    samples_wrote = 0
    counter = 1

    while samples_wrote < samples_total:
        #check if the buffer is not exceeding total samples 
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote

        block = audio[samples_wrote : (samples_wrote + buffer)]
        feat = features(block, sr)
        x = feat.reshape(1, num_rows, num_columns, num_channels)
        prediction = cnn_model.predict(x)
        label_index = np.argmax(prediction,axis=1)[0]
        
        for b in range(len(sample_submission)):
            sample = sample_submission.row_id[b]
            segment_end = counter * 5   
            row_id = sample.split("_")[0] + "_" + sample.split("_")[1] + "_" + sample.split("_")[3]
            target_bird = sample.split("_")[2]
            if labels[label_index] == b:
                target = True
            else:
                target = False
            #data.append([row_id, target])
            pred['row_id'].append(row_id)
            pred['target'].append(True if target > 0.006 else False)
        counter += 1
        samples_wrote += buffer
        
#sample_submission = pd.DataFrame(data, columns=['row_id', 'target'])

In [ ]:
sample_submission = pd.read_csv('../input/birdclef-2022/sample_submission.csv')


for i in range(len(sample_submission)):
    sample = sample_submission.row_id[i]
    key = sample.split("_")[0] + "_" + sample.split("_")[1] + "_" + sample.split("_")[3]
    target_bird = sample.split("_")[2]
    print(key, target_bird)
    if key in prediction_dicts:
        sample_submission.iat[i, 1] = (target_bird in prediction_dicts[key])
sample_submission.to_csv("submission.csv", index=False)
# submission = prediction(test_audios=all_audios,
#                         models_cfg=CFG.models_cfg,
#                         threshold=threshold, 
#                         threshold_long=threshold_long)
# submission.to_csv("submission.csv", index=False)

In [ ]:
results = pd.DataFrame(pred, columns = ['row_id', 'target'])

print(results.head()) 

results.to_csv("submission.csv", index=False)

In [ ]:
#sample_submission.to_csv("/kaggle/working/submission.csv", index=False)

In [ ]:
#sample_submission